# OntoVAE workflow

In [21]:
# import modules
import os
from onto_vae.ontobj import *
from onto_vae.vae_model import *

## 1. Creation of Ontology object

First, we need to initialize an Ontobj. This is required by OntoVAE to initialize latent space and decoder and train the model. The Ontobj stores information about the used ontology, and also datasets that were matched to this ontology so that they can be used for trainind or passed through the VAE model.

In [22]:
# initialize the Ontobj
# the description should be an identifier, e.g. the ontology used, here: PWO (Pathway Ontology)
pwo = Ontobj(description='PWO')

In [79]:
# initialize our ontology
# obo: path to an obo file
# gene_annot: path to a tab separated file with two columns: Genes and Ontology IDs
pwo.initialize_dag(obo=data_path() + 'pw.obo',
                   gene_annot=data_path() + 'gene_term_mapping.txt')

In [4]:
# trim the ontology
pwo.trim_dag(top_thresh=1000, 
             bottom_thresh=30)

In [5]:
# create masks for decoder initialization
pwo.create_masks(top_thresh=1000,
                 bottom_thresh=30)

In [6]:
# match a dataset to the ontology
# expr_path: path to the dataset (either h5ad)
pwo.match_dataset(expr_data = data_path() + 'pbmc_sample_expr.csv',
                  name='PBMC_CD4T')

## 2. OntoVAE model training

In [7]:
# initialize OntoVAE 
pwo_model = OntoVAE(ontobj=pwo,              # the Ontobj we will use
                    dataset='PBMC_CD4T',     # which dataset from the Ontobj to use for model training
                    top_thresh=1000,         # which trimmed version to use
                    bottom_thresh=30)        # which trimmed version to use     
pwo_model.to(pwo_model.device)         

OntoVAE(
  (encoder): Encoder(
    (encoder): ModuleList(
      (0): Sequential(
        (0): Linear(in_features=5892, out_features=228, bias=True)
        (1): BatchNorm1d(228, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Dropout(p=0.2, inplace=False)
        (3): ReLU()
      )
    )
    (mu): Sequential(
      (0): Linear(in_features=228, out_features=228, bias=True)
      (1): Dropout(p=0.5, inplace=False)
    )
    (logvar): Sequential(
      (0): Linear(in_features=228, out_features=228, bias=True)
      (1): Dropout(p=0.5, inplace=False)
    )
  )
  (decoder): OntoDecoder(
    (decoder): ModuleList(
      (0): Sequential(
        (0): Linear(in_features=228, out_features=9, bias=True)
      )
      (1): Sequential(
        (0): Linear(in_features=237, out_features=36, bias=True)
      )
      (2): Sequential(
        (0): Linear(in_features=273, out_features=393, bias=True)
      )
      (3): Sequential(
        (0): Linear(in_features=666, out_fe

In [8]:
# generate a directory where to store the best model
if not os.path.isdir(os.getcwd() + '/models'):
    os.mkdir(os.getcwd() + '/models')

In [9]:
# train the model
pwo_model.train_model(os.getcwd() + '/models/best_model.pt',   # where to store the best model
                     lr=1e-4,                                 # the learning rate
                     kl_coeff=1e-4,                           # the weighting coefficient for the Kullback Leibler loss
                     batch_size=128,                          # the size of the minibatches
                     epochs=5)                                # over how many epochs to train                               # whether run should be logged to Neptune

Epoch 1 of 5


100%|██████████| 2/2 [00:00<00:00, 162.70it/s]


New best model!
Train Loss: 25151.4005
Val Loss: 21928.4121
Epoch 2 of 5


100%|██████████| 2/2 [00:00<00:00, 133.62it/s]


New best model!
Train Loss: 25049.3419
Val Loss: 21895.1714
Epoch 3 of 5


...

New best model!
Train Loss: 24959.0287
Val Loss: 21849.0796
Epoch 4 of 5


100%|██████████| 2/2 [00:00<00:00, 154.34it/s]


New best model!
Train Loss: 24877.0563
Val Loss: 21774.3032
Epoch 5 of 5


100%|██████████| 2/2 [00:00<00:00, 147.71it/s]


New best model!
Train Loss: 24794.9787
Val Loss: 21710.8076


## 3. Analysis with OntoVAE model (pathway activities + perturbations)

We can use a trained OntoVAE model to retrieve pathway activities from latent space and decoder, but also to perform in silico perturbations prior to retrieving the pathway activities.

In [14]:
# load the best model
checkpoint = torch.load(os.getcwd() + '/models/best_model.pt',
                        map_location = torch.device(pwo_model.device), 
                        weights_only=True)
pwo_model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [16]:
# retrieve pathway activities
pwo_act = pwo_model.get_pathway_activities(ontobj=pwo,
                                           dataset='PBMC_CD4T')

We can use a function of the Ontobj to make an example scatterplot for two pathways of our choice.

In [17]:
# make scatterplot for two pathway activities
pwo.plot_scatter(sample_annot = data_path() + 'pbmc_sample_annot.csv',   # pandas Dataframe or path to annotation file
                 color_by = 'condition',                                 # variable to use for coloring
                 act = pwo_act,                                          # pathway activities computed from OntoVAE model
                 term1 = 'interferon mediated signaling pathway',        # term on x-axis of scatter plot
                 term2 = 'T cell receptor signaling pathway',            # term on y-axis of scatter plot
                 top_thresh = 1000,                                      # which trimmed version to use
                 bottom_thresh = 30)                                     # which trimmed version to use

# Note that the scatterplot displayed in the vignette was trained over 200 epochs!

We can now perform in silico perturbations and Wilcoxon tests to see which terms are influenced most.

In [18]:
# get pathway activities where ISG15 was perturbed
pwo_ko_act = pwo_model.perturbation(ontobj=pwo,
                                    dataset='PBMC_CD4T',
                                    genes=['ISG15'],        # list of genes to be perturbed
                                    values=[0])             # list of new values for the genes

We can use a function of the Ontobj to perform a paired Wilcoxon test between perturbed and non-perturbed for all terms of the ontology and get a ranked dataframe

In [19]:
# perform paired Wilcoxon test
results = pwo.wilcox_test(control = pwo_act,
                          perturbed = pwo_ko_act,
                          direction = 'down',
                          top_thresh=1000,
                          bottom_thresh=30)

In [20]:
# display the top results
results.head(10)

,id,term,depth,stat,pval,qval
0,PW:0000024,inflammatory response pathway,0,222197.0,0.001068,0.69224
1,PW:0001116,vitamin A and metabolites signaling pathway,3,224325.0,0.002271,0.69224
2,PW:0000814,Toll-like receptor signaling pathway,5,226636.0,0.004871,0.69224
3,PW:0000817,NOD-like receptor signaling pathway,5,227448.0,0.006281,0.69224
4,PW:0000516,interleukin-6 signaling pathway,6,227895.5,0.007203,0.69224
5,PW:0000541,signaling pathway involving second messengers,0,227999.0,0.007433,0.69224
6,PW:0000233,tumor necrosis factor mediated signaling pathway,6,229194.0,0.010588,0.69224
7,PW:0000106,extrinsic apoptotic pathway,5,229220.5,0.010669,0.69224
8,PW:0000009,apoptotic cell death pathway,4,229222.0,0.010674,0.69224
9,PW:0000650,signaling pathway pertinent to development,0,229924.0,0.013043,0.69224


We see that when we modulate ISG15 expression, our top hits are all pathways related to the immune response!